First all the needed modules are imported. The manipulated toysimulation module is needed.

In [2]:
import numpy as np
import networkx as nx
from collections import defaultdict
import random
import pickle
import matplotlib as mlp
import matplotlib.pyplot as plt
import pandas as pd
import osmnx as ox

import os
import scipy as sp
import scipy.stats

import route

from toysimulations2 import ZeroDetourBus, Stop, Request, Network

ModuleNotFoundError: No module named 'osmnx'

The functions for generating the requests and starting the simulations are defined. In contrast to the functions in jumps_in_request_rate more parameters can be passed to make it possible to start the simulation with an already working bus.

In [3]:
def req_generator_uniform(graph, num_reqs, req_rate,t=0,req_idxi=0):
    """
    Generates requests with rate=req_rate whose origin and
    destination are drawn uniformly randomly. The requests
    are generated in time as a Poisson process.
    """
    req_idx=req_idxi
    

    while req_idx < num_reqs+req_idxi:
        orig, dest = random.sample(graph.nodes(), k=2)
        delta_t = np.random.exponential(1/req_rate)

        t += delta_t
        req_idx += 1
        yield Request(req_idx, t, orig, dest)
        
def simulate_single_request_rate(G, nG, x,network_type, l_avg,initpos=None,time=0,remaining_time=0,next_stop=None,stoplist=[],req_data=dict(),insertion_data=[],req_idx=0):
    """
    Simulates only as single request rate x. See the docstring of
    `simulate_different_request_rates` for details on the arguments.
    Takes in a number of parameters to make it possible to start the simulation with an allready working bus
    """
    num_reqs = 5000
    req_rate = x/2/l_avg
    if initpos==None:
        initpos=random.sample(G.nodes(), k=1)[0]

    sim = ZeroDetourBus(nG,
                        req_generator_uniform(G, num_reqs, req_rate,time,req_idx),
                        network_type, 
                        initpos,
                        time,
                        remaining_time,
                        next_stop,
                        stoplist,
                        req_data,
                        insertion_data
                       )
    print(f"simulating x={x}")
    sim.simulate_all_requests()
    return sim.req_data, sim.insertion_data,sim.stoplist,sim.position,sim.time,sim.remaining_time, sim.next_stop
        

The functions returning the performance parameter data averaged for every request are defined.
One funtion gives back the data for the edge removal, the other for the effect x.
Furthermore, functions that calculate a change in data are defined.

In [5]:
def calculate_avg_cutedge_g(G,nG,l_avg,x,edge):
    
    """
    30 iterations. Averages data over all iterations.
    Starts a simulation with 5000 requests.
    Afterwards an edge is removed and all the data from the first simulation passed to the second simulation.
    """
    wait_gen=[]
    stoplist_gen=[]
    times_gen=[]
    service_gen=[]
    for i in range(0,30):
        req_data1, insertion_data1,stoplist1,position1,time1,remaining_time1, next_stop1=simulate_single_request_rate(G, nG, x, network_type='novolcomp',l_avg=l_avg,initpos=None,time=0,remaining_time=0,next_stop=None,stoplist=[],req_data=dict(),insertion_data=[],req_idx=0)
        
        G.remove_edges_from(edge)
        nG = Network(G,network_type='novolcomp')
        #l_avg = nx.average_shortest_path_length(M)
        req_data, insertion_data,stoplist,position,time,remaining_time, next_stop=simulate_single_request_rate(G, nG, x, network_type='novolcomp',l_avg=l_avg,initpos=position1,time=time1,remaining_time=remaining_time1,next_stop=next_stop1,stoplist=stoplist1,req_data=req_data1,insertion_data=insertion_data1,req_idx=list(req_data1)[-1])
        wait=[]   
        for key in req_data.keys():
            wait.append(float(req_data[key]['pickup_epoch'])-float(req_data[key]['req_epoch']))
        wait_gen.append(wait)
        
        service=[]
        for key in req_data.keys():
            service.append(float(req_data[key]['dropoff_epoch'])-float(req_data[key]['req_epoch']))
        service_gen.append(service)

        stoplist=[]
        for element in insertion_data:
            stoplist.append(element[1])
        stoplist_gen.append(stoplist)  
        
        times_request=[]
        for element in insertion_data:
            times_request.append(element[0])
        times_gen.append(times_request)
        G.add_edges_from(edge)
        nG=Network(G,network_type='novolcomp')
        
    temp_wait=np.array(wait_gen)
    wait_sum=np.transpose(temp_wait)
    wait_avg=[]
    for i in wait_sum:
        wait_avg.append(np.mean(i))
        
    temp_service=np.array(service_gen)
    service_sum=np.transpose(temp_service)
    service_avg=[]
    for i in service_sum:
        service_avg.append(np.mean(i))

    temp_stoplist=np.array(stoplist_gen)
    stoplist_sum=np.transpose(temp_stoplist)
    stoplist_avg=[]
    for element in stoplist_sum:
        stoplist_avg.append(np.mean(element))
        
    temp_times=np.array(times_gen)
    times_sum=np.transpose(temp_times)
    times_avg=[]
    for i in times_sum:
        times_avg.append(np.mean(i))
        
    return wait_avg,stoplist_avg,times_avg,service_avg

def calculate_effect_x_g(G,nG,l_avg,x,edge):
    
    """
    30 iterations. Averages data over all iterations.
    Starts a simulation with 5000 requests.
    Afterwards an edge is removed, the new request rate calculated and the edge is included again.
    Then all the data from the first simulation passed to the second simulation.
    """
    wait_gen=[]
    stoplist_gen=[]
    times_gen=[]
    service_gen=[]
    for i in range(0,30):
        req_data1, insertion_data1,stoplist1,position1,time1,remaining_time1, next_stop1=simulate_single_request_rate(G, nG, x, network_type='novolcomp', l_avg=l_avg,initpos=None,time=0,remaining_time=0,next_stop=None,stoplist=[],req_data=dict(),insertion_data=[],req_idx=0)
        
        G.remove_edges_from(edge)
        nG = Network(G,network_type='novolcomp')
        l_avg_new = nx.average_shortest_path_length(G)
        x_new=(l_avg_new/l_avg)*x
        G.add_edges_from(edge)
        nG = Network(G,network_type='novolcomp')
        req_data, insertion_data,stoplist,position,time,remaining_time, next_stop=simulate_single_request_rate(G, nG, x_new, network_type='novolcomp', l_avg=l_avg,initpos=position1,time=time1,remaining_time=remaining_time1,next_stop=next_stop1,stoplist=stoplist1,req_data=req_data1,insertion_data=insertion_data1,req_idx=list(req_data1)[-1])
        wait=[]   
        for key in req_data.keys():
            wait.append(float(req_data[key]['pickup_epoch'])-float(req_data[key]['req_epoch']))
        wait_gen.append(wait)
        
        service=[]
        for key in req_data.keys():
            service.append(float(req_data[key]['dropoff_epoch'])-float(req_data[key]['req_epoch']))
        service_gen.append(service)

        stoplist=[]
        for element in insertion_data:
            stoplist.append(element[1])
        stoplist_gen.append(stoplist)  
        
        times_request=[]
        for element in insertion_data:
            times_request.append(element[0])
        times_gen.append(times_request)
        
    temp_wait=np.array(wait_gen)
    wait_sum=np.transpose(temp_wait)
    wait_avg=[]
    for i in wait_sum:
        wait_avg.append(np.mean(i))
        
    temp_service=np.array(service_gen)
    service_sum=np.transpose(temp_service)
    service_avg=[]
    for i in service_sum:
        service_avg.append(np.mean(i))

    temp_stoplist=np.array(stoplist_gen)
    stoplist_sum=np.transpose(temp_stoplist)
    stoplist_avg=[]
    for element in stoplist_sum:
        stoplist_avg.append(np.mean(element))
        
    temp_times=np.array(times_gen)
    times_sum=np.transpose(temp_times)
    times_avg=[]
    for i in times_sum:
        times_avg.append(np.mean(i))
        
    return wait_avg,stoplist_avg,times_avg,service_avg

def calculate_two_averages(data,cutoff):
    avg_before=np.mean(data[:cutoff])
    avg_after=np.mean(data[cutoff:])
    return avg_before, avg_after

def calculate_change(data,cutoff):
    avg_before,avg_after=calculate_two_averages(data,cutoff)
    change=avg_after-avg_before
    return change

The data is created on the network for Ingolstadt. Other networks work accordingly. Every edge that can be removed without disconnecting the network is removed and the change in stop list length and wait time calculated. At the same time the change in average shortest path length is calculated and all is saved into a dictionary with the edge as a key.

In [6]:
print("doing Ingolstadt")
graph_path = 'graph_ingolstadt.gpkl'
G = nx.read_gpickle(graph_path)
nG=Network(G,network_type='novolcomp')
l_avg = nx.average_shortest_path_length(G)
x=10
all_edges=G.edges()
edge_change={}
for edge in all_edges:
    print("doing Ingolstadt")
    graph_path = 'graph_ingolstadt.gpkl'
    G = nx.read_gpickle(graph_path)
    nG=Network(G,network_type='novolcomp')
    l_avg = nx.average_shortest_path_length(G)
    x=10
    try:
        edgelist=[]
        edgelist.append(edge)
        wait_avg_e,stoplist_avg_e,times_avg_e,service_avg_e=calculate_avg_cutedge_g(G,nG,l_avg,x,edgelist)
        wait_avg_x,stoplist_avg_x,times_avg_x,service_avg_x=calculate_effect_x_g(G,nG,l_avg,x,edgelist)
        
        change_stoplist_edge_x=calculate_change(stoplist_avg_e_x,5000)
        #print(change_stoplist_edge)
        change_wait_edge_x=calculate_change(wait_avg_e_x,5000)
        #print(change_wait_edge)
       
        change_stoplist_edge=calculate_change(stoplist_avg_e,5000)
        #print(change_stoplist_edge)
        change_stoplist_x=calculate_change(stoplist_avg_x,5000)
        #print(change_stoplist_x)
        change_wait_edge=calculate_change(wait_avg_e,5000)
        #print(change_wait_edge)
        change_wait_x=calculate_change(wait_avg_x,5000)
        #print(change_wait_x)
        #edge_changes_ownnetwork_x10[edge]={'stoplistchange_e':change_stoplist_edge, 'stoplistchange_x':change_stoplist_x, 'waitchange_e':change_wait_edge,'waitchange_x':change_wait_x}
        #print(edge_changes_ingolstadt_x10)
        
        l_avg=nx.average_shortest_path_length(G)
        #print(l_avg)
        G.remove_edges_from(edgelist)
        nG = Network(G,network_type='novolcomp')
        l_avg_change=nx.average_shortest_path_length(G)-l_avg
        #print(l_avg_change)
        G.add_edges_from(edgelist)
        nG = Network(G,network_type='novolcomp')
        

        #print(changes)
        #print(changes_betw)
        edge_change[edge]={'change lavg':l_avg_change,'change stoplist e':change_stoplist_edge,'change stoplist x':change_stoplist_x,'change wait e':change_wait_edge,'change wait x': change_wait_x}
        
    except KeyError:
        G.add_edges_from(edgelist)
        nG = Network(G,network_type='novolcomp')
        print(edge)
        continue
        


doing Ingolstadt


ModuleNotFoundError: No module named 'shapely'

The Hamilton cycle in the remaining network for every single edge is calculated and included in the dictionary.


In [7]:
graph_path = 'graph_ingolstadt.gpkl'
G = nx.read_gpickle(graph_path)
for edge in edge_change.keys():
    edgelist=[]
    edgelist.append(edge)
    G.remove_edges_from(edgelist)
    cycle=nx.approximation.traveling_salesman_problem(G)
    cyclelength=len(cycle)
    G.add_edges_from(edgelist)
    edge_change[edge]['cycle_len']=cyclelength

ModuleNotFoundError: No module named 'shapely'

The Betweenness Centrality for every single edge is calculated and included in the dictionary.

In [8]:
betw_centr=nx.edge_betweenness_centrality(G, k=None, normalized=True, weight=None, seed=None)
for edge in edge_change.keys():
    edge_change['betw cen']=betw_centr[edge]


NameError: name 'G' is not defined

The Betweenness Centrality Concentration in the remaining network for every single edge is calculated and included in the dictionary.

In [9]:
for edge in edge_change.keys():
    edgelist=[]
    edgelist.append(edge)
    G.remove_edges_from(edgelist)
    betweenness_with_all=nx.edge_betweenness_centrality(G, k=None, normalized=True, weight=None,
                                    seed=None)
    total=0
    for i in betweenness_with_all.keys():
        total=total+betweenness_with_all[i]
    hhi=0
    for i in betweenness_with_all.keys():
        hhi=hhi+np.square(betweenness_with_all[i]/total)
    edge_change[edge]['hhi betw']=hhi
    G.add_edges_from(edgelist)

NameError: name 'edge_change' is not defined

The link usage calculated in the file link_usage can also be included in the dictionary.

In [10]:
link_usage=#insert data from file link_usage here
for edge in edge_change.keys():
    edge_change[edge]['link usage']=link_usage_network_1[edge]

SyntaxError: invalid syntax (<ipython-input-10-b02e8d26efb9>, line 1)

From the created dictionary the data can be vizualied.

In [11]:
stoplist_e_ingol=[]
wait_e_ingol=[]
l_avg_changes_ingol=[]
link_usage_ingol=[]
hhi_betw_ingol=[]
stoplist_x_ingol=[]
wait_x_ingol=[]
betw_cen=[]
circle_len=[]

for edge in edge_change.keys():
    link_usage_ingol.append(edge_change[edge]['link usage'])
    l_avg_changes_ingol.append(edge_change[edge]['change lavg'])
    stoplist_e_ingol.append(edge_change[edge]['change stoplist e'])
    wait_e_ingol.append(edge_change[edge]['change wait e'])
    hhi_betw_ingol.append(edge_change[edge]['hhi betw'])
    stoplist_x_ingol.append(edge_change[edge]['change stoplist x'])
    wait_x_ingol.append(edge_change[edge]['change wait x'])
    betw_cen.append(edge_change[edge]['betw cen'])
    circle_len.append(edge_change[edge]['cycle_len'])
    
#example of the visualization of the average shortest path length, other parameters work accordingly
bin_means_e, bin_edges_e, binnumber_e =scipy.stats.binned_statistic(l_avg_changes_ingol, stoplist_e_ingol, statistic='mean', bins=10, range=None)
bin_means_x,bin_edges_x,binnumber_x= scipy.stats.binned_statistic(l_avg_changes_ingol, wait_x_ingol, statistic='mean', bins=10, range=None)

plt.scatter(bin_edges_e[1:], bin_means_e, color='blue',label='Edge')
plt.scatter(bin_edges_x[1:], bin_means_x, color='red',label='Effect x')

plt.xlabel('Change of Average Shortest Path Length')
plt.ylabel('Change in Stop List Length')
plt.title('Ingolstadt')

NameError: name 'changes_ingol' is not defined

For the data of the other networks and the five random networks combined, the same procedure is applied like on the street network of Ingolstadt. To plot five networks into one graph the data from the dictionaries is all united into one array and then plotted. 